In [1]:
import os
import sys
from pprint import pprint
sys.path.append('.')

import pandas as pd
import numpy as np
from tqdm import tqdm
import optuna
from sklearn import metrics
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

from models.gat.gat_pytorch import GAT
from models.gat import params as gat_params
from utils.utils import *
from runners import tools

In [ ]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity

In [2]:
os.chdir('/home/lyz/co-phase-separation/PSGAT/')
DATA_ROOT = './data'

In [ ]:
SAVE_ROOT = './saves/INTEGRATE_ProSE80d_pos1neg1/'

In [ ]:
for i in range(10):
    ppi_path = os.path.join(
        DATA_ROOT,           
        f'PPIN/INTEGRATE.csv'
    )
    edges = pd.read_csv(ppi_path)
    genes = pd.read_pickle(SAVE_ROOT+"genes.pkl")
    emedding = torch.load( os.path.join(SAVE_ROOT, f'embeddings/model_{i}.pt'))
    
    featuremap_cosine = pd.DataFrame(cosine_similarity(emedding))
    featuremap_cosine.columns, featuremap_cosine.index = genes, genes

    
    cosim_list = list()
    for _, x in edges.iterrows():
        cosim = featuremap_cosine[x['A']][x['B']]
        cosim_list.append(cosim)
    edges['cosim'] = cosim_list
    edges.loc[edges['cosim'] < 0, 'cosim'] = 0
    edges.loc[edges['cosim'] > 1, 'cosim'] = 1
    
    featuremap_cosine.to_pickle(os.path.join(SAVE_ROOT, f'pairwise_cosine/model_{i}.pkl'))
    edges.to_pickle(os.path.join(SAVE_ROOT, f'edge_cosine/model_{i}.pkl'))

